<H1> Data Process - Windowing n x n - One Signal</H1>

<h3>Definition of Variables:

In [13]:
signal = "ECG"
sampling_rate_reducer = 100
reading_folder_train = "Train/"
reading_folder_test = "Test/"
writing_folder = "NumPy/" + signal + "/"

<h3>Libraries:

In [2]:
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join
from biosppy import signals as bio_signals

<h3>Functions:

Function Windowing - Segmentation of Data with Windowing

In [3]:
def windowing(df, sampling_rate_reducer):
    new_df = pd.DataFrame(columns = [signal, 'CATEGORY'])

    size = (len(df)-(sampling_rate_reducer*2))
    
    x = 0
    position = 0
    while (position < size) :
        new_df = new_df.append(df[position:position+sampling_rate_reducer])
        position = position+sampling_rate_reducer
        auxiliary = df[position:position+sampling_rate_reducer] 
        delta = auxiliary[signal].mean()
        delta_CATEGORY = df['CATEGORY'][position-sampling_rate_reducer]
        position = position+sampling_rate_reducer
        delta = pd.DataFrame({signal : [delta],'CATEGORY' : [delta_CATEGORY]})
        new_df = new_df.append(delta)
    
    return new_df 

Function Dataset - Convert an Array of Values into a Dataset Matrix

In [4]:
def create_dataset(dataset):
    dataX, dataY = [], []
    i = 0
    
    for i in range(len(dataset)):
        dataX.append([float((dataset.iloc[i:(i + 1), 0]).values)])
        auxiliary = str((dataset.iloc[i:(i + 1), 1]).values)
        auxiliary = auxiliary.replace(auxiliary[0:2],"")
        dataY.append(auxiliary.replace(auxiliary[(len(auxiliary)-2):(len(auxiliary))],""))     
    return dataX, dataY

Function Filter Signal - Performs the Filtering of Signals

In [5]:
def filter_signal(df):
    
    if (signal == "ECG"):
        data = bio_signals.ecg.ecg(signal=df[signal], sampling_rate=1000, show=False)
        new_df = pd.DataFrame(data['filtered'], columns=[signal])    
    elif (signal == "EDA"):
        data = bio_signals.eda.eda(signal=df[signal], sampling_rate=1000, show=False)
        new_df = pd.DataFrame(data['filtered'], columns=[signal])
    elif (signal == "EMG"):
        data = bio_signals.emg.emg(signal=df[signal], sampling_rate=1000, show=False)
        new_df = pd.DataFrame(data['filtered'], columns=[signal])
    
    new_df['CATEGORY'] = df['CATEGORY']
    
    return new_df

Function Read File - Import Data From CSV File

In [ ]:
def read_files(folder):
    file_list = [f for f in listdir(folder) if isfile(join(folder, f))]
    df = pd.DataFrame(columns = ['ECG','EMG','EDA', 'CATEGORY'])
    
    for file in file_list:
        new_df = pd.read_csv(folder + file, sep=';', encoding='utf-8')
        df = df.append(new_df)
    
    return df

Function Save Files - Export Data Processed to a NPY File

In [7]:
def save_files(first_file, second_file, name_first_file, name_second_file):
    np.save(writing_folder + name_first_file, first_file)
    np.save(writing_folder + name_second_file, second_file)

<h3>Data Processing

<h4>Test

In [8]:
df = read_files(reading_folder_test)

In [9]:
x_test = filter_signal(df)

In [10]:
x_test = windowing(x_test, sampling_rate_reducer)

In [11]:
x_test, y_test = create_dataset(x_test[[signal, 'CATEGORY']])

In [12]:
save_files(x_test, y_test,"x_test","y_test")

<h4>Train

In [14]:
df = read_files(reading_folder_train)

In [15]:
x_train = filter_signal(df)

In [16]:
x_train = windowing(x_train, sampling_rate_reducer)

In [17]:
x_train, y_train = create_dataset(x_train[[signal, 'CATEGORY']])

In [18]:
save_files(x_train, y_train,"x_train","y_train")